# Decision Tree From Scratch (Lab 6)
This notebook contains full step‑by‑step implementation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from math import log2


In [ ]:
# Dataset
X, y = make_moons(n_samples=400, noise=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
def gini_impurity(y):
    if len(y)==0: return 0
    _, c = np.unique(y, return_counts=True)
    p = c/len(y)
    return 1 - np.sum(p**2)

def entropy(y):
    if len(y)==0: return 0
    _, c = np.unique(y, return_counts=True)
    p = c/len(y)
    return -np.sum(p*np.log2(p))


In [ ]:
def information_gain(y, y_l, y_r, criterion="gini"):
    imp = gini_impurity if criterion=="gini" else entropy
    H = imp(y)
    n=len(y)
    return H - (len(y_l)/n)*imp(y_l) - (len(y_r)/n)*imp(y_r)


In [ ]:
class TreeNode:
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,value=None):
        self.feature_index=feature_index
        self.threshold=threshold
        self.left=left
        self.right=right
        self.value=value
    def is_leaf(self): return self.value is not None


In [ ]:
class MyDecisionTreeClassifier:
    def __init__(self,max_depth=None,min_samples_leaf=1,criterion="gini"):
        self.max_depth=max_depth
        self.min_samples_leaf=min_samples_leaf
        self.criterion=criterion

    def fit(self,X,y):
        self.root=self._build_tree(X,y,0)

    def _majority(self,y):
        vals,c=np.unique(y,return_counts=True)
        return vals[np.argmax(c)]

    def _build_tree(self,X,y,depth):
        if (self.max_depth and depth>=self.max_depth) or len(np.unique(y))==1 or len(y)<=self.min_samples_leaf:
            return TreeNode(value=self._majority(y))

        best_gain=-1; best_feat=None; best_thr=None
        for f in range(X.shape[1]):
            for t in np.unique(X[:,f]):
                left=X[:,f]<=t
                y_l, y_r = y[left], y[~left]
                gain=information_gain(y,y_l,y_r,self.criterion)
                if gain>best_gain:
                    best_gain=gain; best_feat=f; best_thr=t
        if best_gain<=0:
            return TreeNode(value=self._majority(y))

        left = X[:,best_feat]<=best_thr
        lnode=self._build_tree(X[left], y[left], depth+1)
        rnode=self._build_tree(X[~left], y[~left], depth+1)
        return TreeNode(best_feat,best_thr,lnode,rnode)

    def _predict_one(self,x,node):
        if node.is_leaf(): return node.value
        return self._predict_one(x,node.left if x[node.feature_index]<=node.threshold else node.right)

    def predict(self,X): return np.array([self._predict_one(x,self.root) for x in X])


In [ ]:
tree_gini = MyDecisionTreeClassifier(max_depth=4, criterion="gini")
tree_gini.fit(X_train, y_train)
pred = tree_gini.predict(X_test)
accuracy_score(y_test,pred), confusion_matrix(y_test,pred)
